In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import plotly

df = pd.read_csv("csv/fig_3.csv")
df.head()
# df = df.astype(float)
# df = df.apply(pd.to_numeric, errors='coerce')

In [ ]:
def get_pretty_name(long_name):
    return long_name.replace("_", "")

In [ ]:
cols = plotly.colors.DEFAULT_PLOTLY_COLORS
def get_color(pretty_name):
    if "π0" in pretty_name:
        return cols[0]
    if "π1" in pretty_name:
        return cols[1]
    if "π2" in pretty_name:
        return cols[2]
    if "π3" in pretty_name:
        return cols[3]
    if "π4" in pretty_name:
        return cols[4]
    if "MRU" in pretty_name:
        return cols[5]
    if "Depth first" in pretty_name:
        return cols[6]
    if "Random" in pretty_name:
        return cols[7]
    if "FIFO" in pretty_name:
        return cols[8]
    
    return None


In [ ]:
def get_ci(df_input):
    import math
    stats = df_input.groupby(['p3'], as_index=False)['normalized mean JCT'].agg(['mean', 'count', 'std'])
    # print(stats)
    # print('-'*30)

    ci95_hi = []
    ci95_lo = []

    for i in stats.index:
        p3, m, c, s = stats.loc[i]
        ci95_hi.append(m + 1.96*s/math.sqrt(c))
        ci95_lo.append(m - 1.96*s/math.sqrt(c))

    stats['ci95_hi'] = ci95_hi
    stats['ci95_lo'] = ci95_lo
    # print(stats)
    return stats
# get_ci(df_filtered)

In [ ]:
# Plot a)

template = "plotly_white"
fig = go.Figure()
names = ["MRU",
         "Depth first",
         "Random",
         "FIFO",
         "π_0",
         "π_1",
         "π_2",
         "π_3",
         "π_4"
         ]

names = ["MRU",
         "Depth first",
         "Random",
         "FIFO",
         "π_4"
         ]
for name in names:
    df_filtered = df[df["policy"] == name]
    value_to_plot = "normalized mean JCT"
    # value_to_plot = "latency"
    df_std = df_filtered.groupby("p3", as_index=False)[value_to_plot].std()
    df_mean = df_filtered.groupby("p3", as_index=False)[value_to_plot].mean()
    
    # df_min = df_filtered.groupby("p3", as_index=False)[value_to_plot].min()
    # df_max = df_filtered.groupby("p3", as_index=False)[value_to_plot].max()
    # df_min, df_max = get_ci(df_filtered)
    df_stats = get_ci(df_filtered)
    # df_max = df_mean + df_std
    pretty_name = get_pretty_name(name)
    color = get_color(pretty_name)
    fill_color = plotly.colors.unlabel_rgb(color)
    fill_color = (fill_color[0], fill_color[1], fill_color[2], 0.3)
    fill_color = f"rgba{fill_color}"
    fig.add_trace(go.Scatter(x=df_mean["p3"], y=df_mean[value_to_plot], name=pretty_name, line_color=get_color(pretty_name), line=dict(width=2), marker=dict(size=7),
                             ))
    fig.add_trace(go.Scatter(x=df_stats["p3"], y=df_stats["ci95_hi"], name="upper", line=dict(width=0),
                             showlegend=False, marker=dict(size=0), mode="lines",
                             ))
    fig.add_trace(go.Scatter(x=df_stats["p3"], y=df_stats["ci95_lo"], name="lower",line=dict(width=0),
                             showlegend=False, marker=dict(size=0), mode="lines",
                             fillcolor=fill_color,
                             fill='tonexty',
                             ))
    # break
fig.update_layout(
    title={'text': "a) PPO vs baselines",
           'x': 0.5},
    xaxis_title='p(c)',
    yaxis_title='Normalized JCT',
    # yaxis_range=[0.9, 1.8],
    # yaxis=[rangemode]
    width=8*72,
    height=6*72,
    font=dict(
        # family='Arial bold',
        size=22,
        color='black'
    ),
    legend=dict(
        orientation='h',
        yanchor='top',
        y=1.1,
        xanchor='center',
        x=0.5
    ),
    margin=dict(l=00, r=0, t=60, b=0),
    template=template,
)
pio.show(fig)
pio.write_image(fig, 'img/fig_3_a.png')
pio.write_image(fig, 'pdf/fig_3_a.pdf')
# df_filtered

In [ ]:
# Plot b)

template = "plotly_white"
fig = go.Figure()
names = ["MRU",
         "Depth first",
         "Random",
         "FIFO",
         "π_0",
         "π_1",
         "π_2",
         "π_3",
         "π_4"
         ]

names = ["π_0",
         "π_1",
         "π_2",
         "π_4"
         ]
for name in names:
    df_filtered = df[df["policy"] == name]
    value_to_plot = "normalized mean JCT"
    # value_to_plot = "latency"
    # df_min = df_filtered.groupby("p3", as_index=False)[value_to_plot].min()
    # df_max = df_filtered.groupby("p3", as_index=False)[value_to_plot].max()
    df_mean = df_filtered.groupby("p3", as_index=False)[value_to_plot].mean()
    df_stats = get_ci(df_filtered)
    pretty_name = get_pretty_name(name)
    color = get_color(pretty_name)
    fill_color = plotly.colors.unlabel_rgb(color)
    fill_color = (fill_color[0], fill_color[1], fill_color[2], 0.3)
    fill_color = f"rgba{fill_color}"
    fig.add_trace(go.Scatter(x=df_mean["p3"], y=df_mean[value_to_plot], name=pretty_name, line_color=get_color(pretty_name), line=dict(width=2), marker=dict(size=7),
                             ))
    fig.add_trace(go.Scatter(x=df_stats["p3"], y=df_stats["ci95_hi"], name="upper", line=dict(width=0),
                             showlegend=False, marker=dict(size=0), mode="lines",
                             ))
    fig.add_trace(go.Scatter(x=df_stats["p3"], y=df_stats["ci95_lo"], name="lower",line=dict(width=0),
                             showlegend=False, marker=dict(size=0), mode="lines",
                             fillcolor=fill_color,
                             fill='tonexty',
                             ))
    # break
fig.update_layout(
    title={'text': "b) Fixed vs randomized p(c)",
           'x': 0.5},
    xaxis_title='p(c)',
    yaxis_title='Normalized JCT',
    yaxis_range=[0.9, 1.3],
    # yaxis=[rangemode]
    width=8*72,
    height=6*72,
    font=dict(
        # family='Arial bold',
        size=22,
        color='black'
    ),
    legend=dict(
        orientation='h',
        yanchor='top',
        y=1.1,
        xanchor='center',
        x=0.5
    ),
    margin=dict(l=00, r=0, t=60, b=0),
    template=template,
)
pio.show(fig)
pio.write_image(fig, 'img/fig_3_b.png')
pio.write_image(fig, 'pdf/fig_3_b.pdf')
# df_filtered

In [ ]:
# Plot c)

template = "plotly_white"
fig = go.Figure()
names = ["MRU",
         "Depth first",
         "Random",
         "FIFO",
         "π_0",
         "π_1",
         "π_2",
         "π_3",
         "π_4"
         ]

names = ["π_3",
         "π_4"
         ]
for name in names:
    df_filtered = df[df["policy"] == name]
    value_to_plot = "normalized mean JCT"
    # value_to_plot = "latency"
    df_mean = df_filtered.groupby("p3", as_index=False)[value_to_plot].mean()
    df_stats = get_ci(df_filtered)
    pretty_name = get_pretty_name(name)
    color = get_color(pretty_name)
    fill_color = plotly.colors.unlabel_rgb(color)
    fill_color = (fill_color[0], fill_color[1], fill_color[2], 0.3)
    fill_color = f"rgba{fill_color}"
    fig.add_trace(go.Scatter(x=df_mean["p3"], y=df_mean[value_to_plot], name=pretty_name, line_color=get_color(pretty_name), line=dict(width=2), marker=dict(size=7),
                             ))
    fig.add_trace(go.Scatter(x=df_stats["p3"], y=df_stats["ci95_hi"], name="upper", line=dict(width=0),
                             showlegend=False, marker=dict(size=0), mode="lines",
                             ))
    fig.add_trace(go.Scatter(x=df_stats["p3"], y=df_stats["ci95_lo"], name="lower",line=dict(width=0),
                             showlegend=False, marker=dict(size=0), mode="lines",
                             fillcolor=fill_color,
                             fill='tonexty',
                             ))
    # fig.add_trace(go.Scatter(x=df_max["p3"], y=df_max[value_to_plot], name="upper", line=dict(width=0),
    #                          showlegend=False,
    #                          ))
    # fig.add_trace(go.Scatter(x=df_min["p3"], y=df_min[value_to_plot], name="lower",line=dict(width=0),
    #                          showlegend=False,
    #                          fillcolor=fill_color,
    #                          fill='tonexty',
    #                          ))
    # break
fig.update_layout(
    title={'text': "c) With and without p(c) observation",
           'x': 0.5},
    xaxis_title='p(c)',
    yaxis_title='Normalized JCT',
    yaxis_range=[0.9, 1.3],
    # yaxis=[rangemode]
    width=8*72,
    height=6*72,
    font=dict(
        # family='Arial bold',
        size=22,
        color='black'
    ),
    legend=dict(
        orientation='h',
        yanchor='top',
        y=1.1,
        xanchor='center',
        x=0.5
    ),
    margin=dict(l=00, r=0, t=60, b=0),
    template=template,
)
pio.show(fig)
pio.write_image(fig, 'img/fig_3_c.png')
pio.write_image(fig, 'pdf/fig_3_c.pdf')
# df_filtered